In [1]:
using JLD2
using Statistics
using Printf
using Laplacians
using SparseArrays
using LinearAlgebra

## This notebook outputs the summary tables

In [2]:
function combine(a, b)
    return [a; b]
end

combine (generic function with 1 method)

function extractSlice(dic, slice)
    dicSlice = Dict()
    numepxr = length(dic["ne"])
    for (key, value) in dic
        if length(value) == numepxr
            dicSlice[key] = value[slice]
        else
            dicSlice[key] = value
        end
    end
    return dicSlice
end

In [3]:
# SuiteSparse 
@load "suitesparse_all.jl.split02merge02.jld2"
dic_suitesparse = copy(dic);


In [4]:
function hasZeroDiag(M::SparseMatrixCSC{Tv, Ti}) where {Tv, Ti}
    for i in 1:size(M,1)
        if M[i,i] == zero(Tv)
            return true
        end
    end
    return false
end

function hasZeroDiag(name::String)
    M = downloadSS(name)
    return hasZeroDiag(M)
end

hasZeroDiag (generic function with 2 methods)

In [5]:
function extractSlice(dic, slice)
    dicSlice = Dict()
    numepxr = length(dic["ne"])
    for (key, value) in dic
        if length(value) == numepxr
            dicSlice[key] = value[slice]
        else
            dicSlice[key] = value
        end
    end
    return dicSlice
end

extractSlice (generic function with 1 method)

In [6]:
sizeSlice = findall(dic_suitesparse["ne"] .> 1000)
dic_suitesparse = extractSlice(dic_suitesparse, sizeSlice);

In [7]:
include("../julia-files/downloadSS.jl")
zeroDiagSlice = findall(hasZeroDiag, dic_suitesparse["testName"]);
noZeroDiagSlice = setdiff(1:length(sizeSlice), zeroDiagSlice);

In [8]:
dic_suitesparse = extractSlice(dic_suitesparse, noZeroDiagSlice);

In [9]:
# unweighted chimera
@load "uni_chimera_all.jl.split02merge02.n10000.h1.0.jld2"
dic_uniChimera = copy(dic);
@load "uni_chimera_all.jl.split02merge02.n100000.h2.0.jld2"
dic_uniChimera = merge(combine, dic_uniChimera, dic);
@load "uni_chimera_all.jl.split02merge02.n1000000.h4.0.jld2"
dic_uniChimera = merge(combine, dic_uniChimera, dic);
@load "uni_chimera_all.jl.split02merge02.n10000000.h8.0.jld2"
dic_uniChimera = merge(combine, dic_uniChimera, dic);

In [10]:
# weighted chimera
@load "wted_chimera_nopetsc.jl.split02merge02.n10000.h1.0.jld2"
dic_wtedChimera = copy(dic);
@load "wted_chimera_nopetsc.jl.split02merge02.n100000.h2.0.jld2"
dic_wtedChimera = merge(combine, dic_wtedChimera, dic);
@load "wted_chimera_nopetsc.jl.split02merge02.n1000000.h4.0.jld2"
dic_wtedChimera = merge(combine, dic_wtedChimera, dic);
@load "wted_chimera_nopetsc.jl.split02merge02.n10000000.h8.0.jld2"
dic_wtedChimera = merge(combine, dic_wtedChimera, dic);

In [11]:
# unweighted boundary chimera
@load "uni_bndry_chimera_all.jl.split02merge02.n10000.h1.0.jld2"
dic_uniBndryChimera = copy(dic);
@load "uni_bndry_chimera_all.jl.split02merge02.n100000.h2.0.jld2"
dic_uniBndryChimera = merge(combine, dic_uniBndryChimera, dic);
@load "uni_bndry_chimera_all.jl.split02merge02.n1000000.h4.0.jld2"
dic_uniBndryChimera = merge(combine, dic_uniBndryChimera, dic);
@load "uni_bndry_chimera_all.jl.split02merge02.n10000000.h8.0.jld2"
dic_uniBndryChimera = merge(combine, dic_uniBndryChimera, dic);

In [12]:
# weighted boundary chimera
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n10000.h1.0.jld2"
dic_wtedBndryChimera = copy(dic)
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n100000.h2.0.jld2"
dic_wtedBndryChimera = merge(combine, dic_wtedBndryChimera, dic);
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n1000000.h4.0.jld2"
dic_wtedBndryChimera = merge(combine, dic_wtedBndryChimera, dic);
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n10000000.h8.0.jld2"
dic_wtedBndryChimera = merge(combine, dic_wtedBndryChimera, dic);

In [13]:
dic_chimera = copy(dic_uniChimera);
dic_chimera = merge(combine, dic_chimera, dic_wtedChimera);
dic_chimera = merge(combine, dic_chimera, dic_uniBndryChimera);
dic_chimera = merge(combine, dic_chimera, dic_wtedBndryChimera);

In [14]:
# max flow 
@load "chimeraIPM_nopetsc.jl.split02merge02.jld2"
dic_maxFlow = copy(dic);
@load "spielmanIPM_all.jl.split02merge02.jld2"
dic_maxFlow = merge(combine, dic_maxFlow, dic);

In [15]:
# Sachdeva star 
@load "sachdeva_star_all.jl.split02merge02.jld2"
dic_sachdevaStar = copy(dic);

In [16]:
# SPE 
@load "spe_all.jl.split02merge02.jld2"
dic_spe = copy(dic);

In [17]:
# uniform grid
@load "uniform_grid_all.jl.split02merge02.jld2"
dic_uniGrid = copy(dic);

In [18]:
# checkered grid (high contrast grid)
@load "checkered_all.jl.split02merge02.jld2"
dic_checkeredGrid = copy(dic);

In [19]:
# aniso grid 
@load "aniso_all.jl.split02merge02.jld2"
dic_anisoGrid = copy(dic);

In [20]:
# weighted grid 
@load "wgrid_all.jl.split02merge02.jld2"
dic_wtedGrid = copy(dic);

In [21]:
dic_grid = copy(dic_uniGrid);
dic_grid = merge(combine, dic_grid, dic_checkeredGrid);
dic_grid = merge(combine, dic_grid, dic_anisoGrid);
dic_grid = merge(combine, dic_grid, dic_wtedGrid);

### done

In [31]:
# dicAll = [dic_suitesparse, dic_uniChimera, dic_wtedChimera, dic_uniBndryChimera, dic_wtedBndryChimera, dic_maxFlow, dic_sachdevaStar, dic_spe, dic_uniGrid, dic_checkeredGrid,dic_anisoGrid,dic_wtedGrid]
dicAll = [dic_suitesparse, dic_spe, dic_grid, dic_chimera, dic_sachdevaStar]

5-element Vector{Dict{Any, Any}}:
 Dict("acM2C2_err" => [3.2982371685298304e-9, 6.348186189107466e-9, 2.7873143159556726e-15, 4.87683398093098e-9, 5.985058421275326e-9, 8.803790029556267e-9, 1.9287984468639396e-16, 1.4949753227936238e-15, 1.4802743372104384e-15, 2.6437327820944332e-9  …  3.1970041164591543e-9, 5.579236545376999e-9, 5.932099317095519e-16, 3.4828607954007058e-9, 5.810373233705067e-9, 4.392074968504867e-9, 2.1978023095678265e-16, 5.355500047190546e-9, 4.5564439954315e-9, 8.840361962792183e-9], "hypre_solve" => [0.05999999, 0.03, 0.0, 0.0, 0.03, 0.009999998, 0.0, 0.0, 0.0, 0.0  …  0.12, 0.03, 0.0, 0.04, 0.0, 0.07, 0.0, 1.07, 0.009999998, 0.14], "acM2C2_solve" => [0.0757741928100586, 0.0490109920501709, 0.00023412704467773438, 0.013978004455566406, 0.049938201904296875, 0.001210927963256836, 0.00023603439331054688, 0.00023603439331054688, 0.00023794174194335938, 0.00981903076171875  …  0.18432283401489258, 0.038838863372802734, 0.0002639293670654297, 0.050592899322509766, 0

In [33]:
# instanceNames = ["SuiteSparse Matrix Collection",  "Unweighted Chimeras", "Weighted Chimeras", "Unweighted SDDM Chimeras", "Weighted SDDM Chimeras", "Maximum flow IPMs", "Sachdeva stars", "SPE benchmark", "Uniform coefficient Poisson grid", "High contrast coefficient Poisson grid", "Anisotropic coef. Poisson grid, variable discretization", "Anisotropic coef. Poisson grid, variable weight"];
instanceNames = ["SuiteSparse Matrix Collection", "SPE benchmark", "Poisson grid problems", "Chimeras", "Sachdeva stars"]

5-element Vector{String}:
 "SuiteSparse Matrix Collection"
 "SPE benchmark"
 "Poisson grid problems"
 "Chimeras"
 "Sachdeva stars"

In [34]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.4g" sd(3,s)*1e6)

ffe6 (generic function with 1 method)

In [35]:
solvers=["ac", "ac-s2m2", "cmg2", "hypre", "petsc_hypre",  "icc2"];

In [36]:
function get_tvn_max(d, solvers)
    
    results_array = []
    for solver in solvers 
        if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
            cur_err = maximum(d[string(solver, "_err")])
            tvn = maximum((d[string(solver, "_tot")] ./ d["ne"]))
            if cur_err <= 1e-8
                push!(results_array, (solver, ffe6(tvn)))
            elseif cur_err < 1e-4
                push!(results_array, (solver, string(ffe6(tvn), "\\tnote{*}")))
            elseif cur_err < 1
                push!(results_array, (solver, string(ffe6(tvn), "\\tnote{**}")))
            else
                push!(results_array, (solver, "Inf"))
            end
        else
            push!(results_array, (solver, "N/A"))
        end
    end

    results = Dict(results_array)
    return results
end

get_tvn_max (generic function with 1 method)

In [37]:
for (index, d) in enumerate(dicAll)
    max_tvns = get_tvn_max(d, solvers)
    line = string("{", instanceNames[index], "}&", join([max_tvns[solver] for solver in solvers], "&"), "\\\\")
    println(line)
end

{SuiteSparse Matrix Collection}&0.994&1.4&371&Inf&27.1&1450\tnote{*}\\
{SPE benchmark}&1.62&1.91&1.1&0.648&2.58&6.13\\
{Poisson grid problems}&2.33&3.57&1.27&0.772&4.92&6.56\\
{Chimeras}&4.39&7.19&Inf&Inf&N/A&111\tnote{*}\\
{Sachdeva stars}&4.76\tnote{**}&1.04&1.18&3.52\tnote{**}&7.22\tnote{**}&1.02\\


## CMG problems

In [ ]:
cmg_failures = []
for (index, d) in enumerate(dicAll)
    for i in 1:length(d["testName"])
        if d["cmg_err"][i] == 1.0 || d["cmg_err"][i] == Inf
            push!(cmg_failures, d["testName"][i])
        end
    end
end


In [ ]:
cmg_failures

21-element Vector{Any}:
 "HB/bcsstm23"
 "HB/bcsstm24"
 "HB/bcsstm09"
 "Boeing/bcsstm39"
 "HB/bcsstm25"
 "Oberwolfach/t3dl_e"
 "HB/bcsstm11"
 "uni_chimera(100000,51)"
 "uni_chimera(100000,69)"
 "uni_chimera(1000000,17)"
 "wted_chimera(100000,51)"
 "wted_chimera(100000,69)"
 "wted_chimera(1000000,6)"
 "wted_chimera(1000000,17)"
 "uni_bndry_chimera(100000,51)"
 "uni_bndry_chimera(1000000,12)"
 "uni_bndry_chimera(1000000,17)"
 "uni_bndry_chimera(1000000,28)"
 "uni_bndry_chimera(1000000,29)"
 "uni_bndry_chimera(10000000,11)"
 "wted_bndry_chimera(1000000,17)"

In [ ]:
cmg2_failures = []
for (index, d) in enumerate(dicAll)
    for i in 1:length(d["testName"])
        if d["cmg2_err"][i] == 1.0 || d["cmg2_err"][i] == Inf
            push!(cmg2_failures, d["testName"][i])
        end
    end
end

In [ ]:
cmg2_failures

23-element Vector{Any}:
 "uni_chimera(100000,13)"
 "uni_chimera(100000,34)"
 "uni_chimera(100000,35)"
 "uni_chimera(100000,66)"
 "uni_chimera(100000,72)"
 "uni_chimera(10000000,6)"
 "wted_chimera(100000,13)"
 "wted_chimera(100000,35)"
 "wted_chimera(100000,66)"
 "wted_chimera(100000,72)"
 "wted_chimera(10000000,6)"
 "uni_bndry_chimera(100000,13)"
 "uni_bndry_chimera(100000,35)"
 "uni_bndry_chimera(100000,66)"
 "uni_bndry_chimera(100000,72)"
 "uni_bndry_chimera(10000000,6)"
 "wted_bndry_chimera(100000,13)"
 "wted_bndry_chimera(100000,35)"
 "wted_bndry_chimera(100000,66)"
 "wted_bndry_chimera(100000,72)"
 "wted_bndry_chimera(10000000,6)"
 "chimeraIPM.n100000.i5.eps0.1.1"
 "chimeraIPM.n100000.i5.eps0.1.4"

## ICC Failure

In [ ]:
icc_failures = []
for (index, d) in enumerate(dicAll)
    for i in 1:length(d["testName"])
        if d["icc_err"][i] == 1.0 || d["icc_err"][i] == Inf
            push!(icc_failures, d["testName"][i])
        end
    end
end


In [ ]:
icc_failures

71-element Vector{Any}:
 "uni_chimera(10000,25)"
 "uni_chimera(10000,51)"
 "uni_chimera(10000,52)"
 "uni_chimera(100000,42)"
 "uni_chimera(100000,53)"
 "uni_chimera(1000000,13)"
 "uni_chimera(1000000,17)"
 "wted_chimera(10000,25)"
 "wted_chimera(10000,51)"
 "wted_chimera(10000,52)"
 "wted_chimera(100000,42)"
 "wted_chimera(100000,53)"
 "wted_chimera(1000000,13)"
 ⋮
 "ipmR.k600.low0.25.up1.0e-6.i6"
 "ipmR.k600.low0.25.up1.0e-6.i7"
 "ipmR.k600.low0.25.up1.0e-6.i8"
 "ipmR.k600.low0.25.up1.0e-6.i9"
 "ipmR.k600.low0.25.up1.0e-6.i10"
 "ipmR.k600.low0.25.up1.0e-6.i11"
 "star_join(complete_graph(100), 50)"
 "star_join(complete_graph(150), 75)"
 "star_join(complete_graph(350), 175)"
 "star_join(complete_graph(500), 250)"
 "star_join(complete_graph(700), 350)"
 "star_join(complete_graph(800), 400)"

In [ ]:
icc2_failures = []
for (index, d) in enumerate(dicAll)
    for i in 1:length(d["testName"])
        # if d["icc2_err"][i] == 1.0 || d["icc2_err"][i] == Inf
        if d["icc2_err"][i] > 1e-8
            push!(icc2_failures, d["testName"][i])
        end
    end
end


In [ ]:
icc2_failures

119-element Vector{Any}:
 "MaxPlanck/shallow_water2"
 "uni_chimera(10000,10)"
 "uni_chimera(10000,16)"
 "uni_chimera(10000,56)"
 "uni_chimera(10000,63)"
 "uni_chimera(100000,2)"
 "uni_chimera(100000,4)"
 "uni_chimera(100000,15)"
 "uni_chimera(100000,20)"
 "uni_chimera(100000,39)"
 "uni_chimera(100000,53)"
 "uni_chimera(100000,61)"
 "uni_chimera(100000,88)"
 ⋮
 "ipmR.k500.low0.25.up1.0e-6.i6"
 "ipmR.k500.low0.25.up1.0e-6.i7"
 "ipmR.k500.low0.25.up1.0e-6.i10"
 "ipmR.k600.low0.25.up1.0e-6.i3"
 "ipmR.k600.low0.25.up1.0e-6.i4"
 "ipmR.k600.low0.25.up1.0e-6.i5"
 "ipmR.k600.low0.25.up1.0e-6.i6"
 "ipmR.k600.low0.25.up1.0e-6.i7"
 "ipmR.k600.low0.25.up1.0e-6.i8"
 "ipmR.k600.low0.25.up1.0e-6.i9"
 "ipmR.k600.low0.25.up1.0e-6.i10"
 "ipmR.k600.low0.25.up1.0e-6.i11"

In [ ]:
M = uni_bndry_chimera(100000,35);
b = randn(size(M,1))
b = M * b
b = b ./ norm(b);


In [ ]:
include("../julia-files/matlabSafe.jl")

timeLimitRchol (generic function with 1 method)

In [ ]:
timeLimitIcc(Inf, M, b)


                            < M A T L A B (R) >
                  Copyright 1984-2024 The MathWorks, Inc.
                  R2024a (24.1.0.2537033) 64-bit (glnxa64)
                             February 21, 2024

 
For online documentation, see https://www.mathworks.com/support
For product information, visit www.mathworks.com.
 
>> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> 
relres =

   3.8015e-09

>> >> >> >> >> 

(1.202958, 0.970358, 38, 3.801525468639023e-9)

In [ ]:
for d in dicAll
    if maximum(d["icc2_err"]) > 1e-8
        println("Found a failure")
        @show d["testName"][argmax(d["icc2_err"])]
        @show maximum(d["icc2_err"])
    end
end

Found a failure
(d["testName"])[argmax(d["icc2_err"])] = "MaxPlanck/shallow_water2"
maximum(d["icc2_err"]) = 5.507527219983869e-7
Found a failure
(d["testName"])[argmax(d["icc2_err"])] = "uni_chimera(10000000,7)"
maximum(d["icc2_err"]) = 2.7725927312355067e-5
Found a failure
(d["testName"])[argmax(d["icc2_err"])] = "wted_chimera(10000000,7)"
maximum(d["icc2_err"]) = 2.7725927312355067e-5
Found a failure
(d["testName"])[argmax(d["icc2_err"])] = "wted_bndry_chimera(100000,35)"
maximum(d["icc2_err"]) = 2.3714644750974205e-7
Found a failure
(d["testName"])[argmax(d["icc2_err"])] = "chimeraIPM.n100000.i1.eps0.01.5"
maximum(d["icc2_err"]) = 7.818428824773771e-7


In [ ]:
a = uni_chimera(10000000,7)
c = components(a)
@show length(unique(c))

length(unique(c)) = 1


1

In [ ]:
include("../julia-files/downloadSS.jl")
include("../julia-files/isLap.jl")
include("../julia-files/compareSolvers.jl")
include("../julia-files/matlabSafe.jl")

timeLimitRchol (generic function with 1 method)

In [ ]:
M = downloadSS("McRae/ecology1")
a, _ = adj(M)
c = components(a)
@show length(unique(c))

b = randn(size(M,1))
b = M * b
b = b ./ norm(b);

length(unique(c)) = 1


In [ ]:
isLap(M)

true

In [ ]:
# timeLimitIcc(Inf, M, b)
timeLimitIccLap(Inf, M, b)

cmd = `timeout Inf /home/yuagao/matlab_r2024a/bin/matlab -nojvm -nodisplay -nosplash -nodesktop '<' /home/yuagao/SDDM2023/matlab/timeIccLap.m`

                            < M A T L A B (R) >
                  Copyright 1984-2024 The MathWorks, Inc.
                  R2024a (24.1.0.2537033) 64-bit (glnxa64)
                             February 21, 2024

 
For online documentation, see https://www.mathworks.com/support
For product information, visit www.mathworks.com.
 
>> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> 
relres =

   1.1351e-07

>> >> >> >> >> 

(77.371018, 0.586211, 995, 1.1351287904864473e-7)

In [ ]:
dic_test = Dict()

testLap([], dic_test, M, b; test_icc=true)

cmd = `timeout 18000 /home/yuagao/matlab_r2024a/bin/matlab -nojvm -nodisplay -nosplash -nodesktop '<' /home/yuagao/SDDM2023/matlab/timeIccCon.m`

                            < M A T L A B (R) >
                  Copyright 1984-2024 The MathWorks, Inc.
                  R2024a (24.1.0.2537033) 64-bit (glnxa64)
                             February 21, 2024

 
For online documentation, see https://www.mathworks.com/support
For product information, visit www.mathworks.com.
 
>> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> >> >> >> >> >> >> >> >> >> >> >> >> >> >> >> 
relres =

   3.2174e-09

>> >> >> >> >> Build Time: 0.158572
Solve Time: 0.187478
Iterations: 10
error: 3.217365459449211e-9
Time to load and start Matlab: 8038 milliseconds/1000)


Any[]